# Impact Quantization

In [1]:
# imports
import os
import numpy as np
import pandas as pd
import pickle

## Functions

In [2]:
# function to quantize impact points
def quantize_impact_points(impact_points, time_stamps, quant_params):
    assert len(impact_points) == len(time_stamps)
    n_frames, n_limbs = impact_points.shape
    
    # quantize time stamps
    dt = 60 / (quant_params['bpm'] * quant_params['subdiv'])
    quantized_time_stamps = np.round((time_stamps - quant_params['beat_ref']) / dt) * dt + quant_params['beat_ref']
    impact_points[quantized_time_stamps < 0] = np.zeros((n_limbs), dtype=bool)
    quantized_time_stamps[quantized_time_stamps < 0] = 0
    
    # format quantized impacts
    quantized_impacts = []
    for l in range(n_limbs):
        quantized_impacts.append(np.unique(quantized_time_stamps[impact_points[:,l]]))
    
    return quantized_impacts

## Quantize Impact Points

In [3]:
# load impact points
load_dir = os.path.join('..', 'data', 'impact_points')
vid_name = 'j2'
impact_points = np.load(os.path.join(load_dir, f'{vid_name}_impact_points.npy'))
time_stamps = np.load(os.path.join(load_dir, f'{vid_name}_time_stamps.npy'))
print(f'Percentage impacts per limb pre quant: {100 * impact_points.mean(axis=0)}')

# quant params for j2
quant_params = {
    'beat_ref': 2.4,
    'bpm': 159,
    'subdiv': 2
}

# run
quantized_impacts = quantize_impact_points(impact_points, time_stamps, quant_params)

# save results to file
save_dir = os.path.join('..', 'data', 'quantized_impacts')
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

with open(os.path.join(save_dir, f'{vid_name}_quantized_impacts.pkl'), 'wb') as f:
    pickle.dump(quantized_impacts, f)
quantized_impacts
print(f'Percentage impacts per limb post quant: {100 * np.array([len(x) for x in quantized_impacts]) / impact_points.shape[0]}')

Percentage impacts per limb pre quant: [2.2556391  1.94235589 2.38095238 2.19298246]
Percentage impacts per limb post quant: [2.2556391  1.94235589 2.38095238 2.19298246]


## Run on All Videos

In [4]:
# run impact detection for all videos
load_dir = os.path.join('..', 'data', 'impact_points')
vid_info_df = pd.read_csv(os.path.join('..', 'video', 'video_info.csv'), index_col='vid_name')

save_dir = os.path.join('..', 'data', 'quantized_impacts')
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# iterate over all videos
for vid_name in vid_info_df.index:
    if vid_name != 'group':
        print(10 * '-')
        print(vid_name)
        
        # load impact points and time stamps
        impact_points = np.load(os.path.join(load_dir, f'{vid_name}_impact_points.npy'))
        time_stamps = np.load(os.path.join(load_dir, f'{vid_name}_time_stamps.npy'))
         
        # quantize impact points and save
        quant_params = {
            'beat_ref': vid_info_df.loc[vid_name]['beat_ref'],
            'bpm': vid_info_df.loc[vid_name]['bpm'],
            'subdiv': vid_info_df.loc[vid_name]['subdiv']
        }
        quantized_impacts = quantize_impact_points(impact_points, time_stamps, quant_params)
        with open(os.path.join(save_dir, f'{vid_name}_quantized_impacts.pkl'), 'wb') as f:
            pickle.dump(quantized_impacts, f)

----------
a1
----------
a2
----------
e1
----------
e2
----------
e3
----------
j1
----------
j2
----------
u1
----------
u2
